# Wind Turbine Farm Management Assistant

## Objective

This notebook demonstrates the following:

- Utilizing Assistant tools such as the Code Interpreter and Function calling, this bot is capable of retrieving a CSV file that illustrates turbine wind speed, voltage, and the last maintenance date. It assists you in reviewing the file contents and aids in determining whether a specific turbine is in need of maintenance.

This tutorial uses the following Azure AI services:
- Access to Azure OpenAI Service - you can apply for access [here](https://aka.ms/oai/access)
- Azure OpenAI service - you can create it from instructions [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource)
- Azure OpenAI Studio - go to [https://oai.azure.com/](https://oai.azure.com/) to work with the Assistants API Playground
- A connection to the Azure OpenAI Service with a [Key and Endpoint](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart)

Reference:
- Learn more about how to use Assistants with our [How-to guide on Assistants](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant).
- [Assistants OpenAI Overview](https://platform.openai.com/docs/assistants/overview)


## Time

You should expect to spend 5-10 minutes running this sample.

## About this example

This sample demonstrates the creation of an Azure OpenAI Assistant named "Turbine Management Assistant" utilizing the Azure OpenAI API. The assistant functions as a management tool for wind turbine farms, providing data, charts, and actionable insights regarding the status and performance of each turbine within the farm. The notebook/script orchestrates a dialogue with the assistant, navigating through different inquiries and scenarios to demonstrate its capabilities.

### Data
This sample uses files from the folder [`data/`](./data/) in this repo. You can clone this repo or copy this folder to make sure you have access to these files when running the sample.


## Before you begin



### Installation

Install the following packages required to execute this notebook. 



In [ ]:
# Install the packages
%pip install -r ../requirements.txt

### Parameters

In [34]:
import os

# from dotenv import load_dotenv

# load_dotenv("../.env")

api_endpoint = ""
api_key = ""
api_version = "2024-02-15-preview"
api_deployment_name = "bp1"
email_URI = os.getenv("EMAIL_URI")

should_cleanup: bool = False

## Run this Example

### Load the required libraries

In [35]:
import io
import time
from datetime import datetime
from pathlib import Path
from typing import Iterable

from openai import AzureOpenAI
from openai.types import FileObject
from openai.types.beta.threads.message_content_image_file import MessageContentImageFile
from openai.types.beta.threads.message_content_text import MessageContentText
from openai.types.beta.threads.messages import MessageFile
from PIL import Image

### Create an AzureOpenAI client

In [64]:
client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

### Define the Assistant tools

In [65]:
tools_list = [
    {"type": "code_interpreter"},
]

### Upload the Assistant file(s)

In [66]:
DATA_FOLDER = "data/"


def upload_file(client: AzureOpenAI, path: str) -> FileObject:
    with Path(path).open("rb") as f:
        return client.files.create(file=f, purpose="assistants")


arr = os.listdir(DATA_FOLDER)

print(arr)
assistant_files = []
for file in arr:
    filePath = DATA_FOLDER + file
    assistant_files.append(upload_file(client, filePath))

file_ids = [file.id for file in assistant_files]

print(file_ids)

['input.cs', 'logging.md']
['assistant-AttsuylMMXaexNQFgWih5uCR', 'assistant-Sy5i31CCwWjjPgcYr97ZX7Zf']


### Create an Assistant and a Thread

In [67]:
assistant = client.beta.assistants.create(
    name="C# .NET string ILogger logging best practice enforcer.",
    instructions="You are a programming master that can read technical documentations and rewrite C# .NET code following the best practices from the the technical documentation. You will be given some code where logging were not done following the best practices. You will help to refactor the code following the best practice based on the technical documentation you read.",
    tools=tools_list,
    model="yunlModel",
    file_ids=file_ids,
)

thread = client.beta.threads.create()
print(thread.id)

thread_Wk85v2qLicgj7GJXsybE83nF


### Format and display the Assistant Messages for text and image

In [69]:
def format_messages(messages: Iterable[MessageFile]) -> None:
    message_list = []

    # Get all the messages till the last user message
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break

    # Reverse the messages to show the last user message first
    message_list.reverse()

    # Print the user or Assistant messages or images
    for message in message_list:
        for item in message.content:
            # Determine the content type
            if isinstance(item, MessageContentText):
                print(f"{message.role}:\n{item.text.value}\n")
            elif isinstance(item, MessageContentImageFile):
                # Retrieve image from file id
                response_content = client.files.content(item.image_file.file_id)
                data_in_bytes = response_content.read()
                # Convert bytes to image
                readable_buffer = io.BytesIO(data_in_bytes)
                image = Image.open(readable_buffer)
                # Resize image to fit in terminal
                width, height = image.size
                image = image.resize((width // 2, height // 2), Image.LANCZOS)
                # Display image
                image.show()

### Process the user messages

In [70]:
def process_message(content: str) -> None:
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=content)

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="The current date and time is: " + datetime.now().strftime("%x %X") + ". ",
    )

    print("processing...")
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == "completed" or run.status == "failed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            format_messages(messages)
            break
        if run.status == "expired":
            # Handle expired
            break
        if run.status == "cancelled":
            # Handle cancelled
            break
        if run.status == "requires_action":
            pass
        else:
            # print("in progress...",run.status)
            time.sleep(5)

###  Have a conversation with the Assistant

In [10]:
question = "You are in the right direction! Now, can you rewrite all occurences that utilizes in the input.cs file I uploaded in the data folder with the more performant way defined in the Program.cs file I uploaded? Please write the updated version of input.cs into a new file called output.cs."
process_message(question)

processing...


KeyboardInterrupt: 

### More questions


In [71]:
question = "In the input.cs file, find all the occurrences that are using C# .`ILogger` interface for logging. Next, identify which of these occurrences use string interpolation for logging. For those that are using string interpolation, replace them with structured logging using message templates. Write the updated content with the updated logging code to a new file."
process_message(question)



processing...
user:
In the input.cs file, find all the occurrences that are using C# .`ILogger` interface for logging. Next, identify which of these occurrences use string interpolation for logging. For those that are using string interpolation, replace them with structured logging using message templates. Write the updated content with the updated logging code to a new file.

assistant:
To address your request, I'll follow these steps:

1. Identify the file named `input.cs` from the uploaded files. The file may have been uploaded with a different name, so I will start by listing the uploaded files.
2. Read the contents of the `input.cs` file.
3. Find all occurrences of the C# `ILogger` interface being used for logging.
4. Identify which of these occurrences use string interpolation.
5. Replace the string interpolation with structured logging using message templates.
6. Write the updated content to a new file.

Let's start by listing the two files you uploaded to identify which one is 

In [45]:
question = "After learning logging best practices recommended in the logging.md file, please refactor input.cs file in the data folder and output a new file as a result: specifically, use log message template instead of string interpolation when possible."
process_message(question)

processing...
user:
After learning logging best practices recommended in the logging.md file, please refactor input.cs file in the data folder and output a new file as a result: specifically, use log message template instead of string interpolation when possible.

assistant:
To start with, I'll first look at the contents of the `logging.md` file, which you have uploaded to understand the logging best practices that you are referring to. Then, I'll refactor the `input.cs` file accordingly and save the output to a new file.

Let's begin by reading the contents of the `logging.md` file.

assistant:
It seems that the output has been truncated and therefore doesn't contain direct reference to the logging best practices you mentioned. However, based on the context, the logging best practices probably involve using structured logging with message templates as opposed to string interpolation, to allow better searching and filtering of log data.

Now, let's read the contents of the `input.cs` f

In [46]:
question = "Yes, you are in the right direction! Please proceed in refactoring the input.cs file."
process_message(question)

processing...
user:
Yes, you are in the right direction! Please proceed in refactoring the input.cs file.

assistant:
Thank you for the confirmation. Based on the discussion, we identified one example of string interpolation in a logging statement, which I'll now refactor to use the log message template format:

```csharp
logger.LogInformation($"{{{{Number}}}}: {{Number}}", 5);
```

This line incorrectly demonstrates string interpolation, but it should be refactored to use the structured logging approach. It will be changed to:

```csharp
logger.LogInformation("{Number}: {Value}", "Number", 5);
```

Given that there might be multiple instances and variants of such logging statements in the `input.cs` file, I will programmatically refactor all instances that match the pattern and output the refactored `input.cs` file to a new file.

Let's perform the refactoring now.

assistant:
It appears there was a misunderstanding in the execution of the refactoring. The function I implemented is ba

## Cleaning up


In [32]:
if should_cleanup:
    client.beta.assistants.delete(assistant.id)
    client.beta.threads.delete(thread.id)
    for file in assistant_files:
        client.files.delete(file.id)

NotFoundError: Error code: 404 - {'error': {'message': 'No such File object: assistant-qVA82cZ16T2E7bKU6ObCVv6v', 'type': 'invalid_request_error', 'param': 'id', 'code': None}}